In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tensorflow.python.ops import rnn, rnn_cell
from yahoo_finance import Share

In [2]:
day_len = 9
H = 256 # number of hidden layer neurons
D = 9 # input dimensionality
C = 1 # ouput classes 
n_input = 3 # MNIST data input (img shape: 2*2)
n_steps = 3 # timesteps
n_day = 20

### get 2330 stock data from yahoo-finance

In [3]:
# get stock from yahoo-finance
stock = Share('2317.TW')
today = datetime.date.today()
stock_data = stock.get_historical('2013-01-01', str(today))
stock_data.reverse()
print len(stock_data)

1025


### delete the zero-volumn data

In [4]:
i = 0
while( i < len(stock_data)):
    if (int(stock_data[i].get('Volume')) <= 0):
        stock_data.remove(stock_data[i])
        i = -1
    i += 1
print 'data without zero volumn', len(stock_data)

data without zero volumn 959


### training and testing data

In [5]:
data_X = np.zeros((len(stock_data)-day_len,day_len), dtype=np.float)
data_Y = np.zeros((len(stock_data)-day_len,1), dtype = np.float)

for i in xrange(0, len(data_X)):
    for j in xrange(0, day_len):
        data_X[i,j] = float(stock_data[i+j].get('Close'))
        
for i in xrange(0, len(data_X)):
    tmp = []
    for j in xrange(0, day_len):
        tmp.append(data_X[i,j])
    for j in xrange(0,day_len):
        data_X[i,j] = (data_X[i,j]-min(tmp))/(max(tmp)-min(tmp))

# rise in n_day or not? 1:rise, 0:otherwise
for i in xrange(0, len(data_X)-1-n_day):
    for j in xrange(i+day_len+1, i+day_len+1+n_day):
        if stock_data[j].get('Close') > stock_data[i+day_len].get('Close'):
            data_Y[i] = 1
            break

In [6]:
len_train = int(len(data_X) * 0.8)
len_test = len(data_X) - len_train

train_X = data_X[0:len_train]
train_Y = data_Y[0:len_train]
test_X = data_X[len_train::]
test_Y = data_Y[len_train::]

In [7]:
# LSTM
# placeholder
input_x = tf.placeholder(tf.float32, [None,D], name="input_x")
input_y = tf.placeholder(tf.float32, [None,C], name="input_y")

# weights and biases
w1 = tf.get_variable("W1", shape=[H, C], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.constant(0.01, shape = [C], name = "B1"))

input_x1 = tf.reshape(input_x, [-1,n_steps, n_input])
input_x2 = tf.transpose(input_x1, [1, 0, 2])
input_x3 = tf.reshape(input_x2, [-1, n_input])
input_x4 = tf.split(0, n_steps, input_x3)

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=1.0)
outputs, states = rnn.rnn(lstm_cell, input_x4, dtype=tf.float32)
score = tf.matmul(outputs[-1], w1) + b1

loss = tf.reduce_mean(tf.reduce_sum(tf.square(score-input_y),reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss) 
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss) 

In [8]:
init = tf.initialize_all_variables()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(5000):
        
        # training
        sess.run(optimizer, feed_dict={input_x: train_X, input_y: train_Y})
        
        if i % 50 == 0:
            
            print i
            print 'loss:', sess.run(loss, feed_dict={input_x:train_X, input_y: train_Y})

            train_prediction = sess.run(score, feed_dict={input_x: train_X})
            for i in xrange(0, len(train_prediction)):
                if train_prediction[i] >= 0.5:
                    train_prediction[i] = 1
                else:
                    train_prediction[i] = 0
                
            test_prediction = sess.run(score, feed_dict={input_x: test_X})
            for i in xrange(0, len(test_prediction)):
                if test_prediction[i] >= 0.5:
                    test_prediction[i] = 1
                else:
                    test_prediction[i] = 0
                    
            train_acc = 0
            test_acc = 0

            for i in xrange(0, len(train_prediction)):
                #train_prediction[i] = round(train_prediction[i])
                if train_prediction[i] == train_Y[i]:
                    train_acc = train_acc + 1
            train_acc = float(train_acc) / float(len(train_prediction))
        
            for i in xrange(0, len(test_prediction)):
                #test_prediction[i] = round(test_prediction[i])
                if test_prediction[i] == test_Y[i]:
                    test_acc = test_acc + 1
            test_acc = float(test_acc) / float(len(test_prediction))
            
            print 'training acc=', train_acc
            print 'testing acc=', test_acc

0
loss: 0.646797
training acc= 0.118421052632
testing acc= 0.242105263158
50
loss: 0.116684
training acc= 0.878947368421
testing acc= 0.752631578947
100
loss: 0.11062
training acc= 0.881578947368
testing acc= 0.757894736842
150
loss: 0.107632
training acc= 0.881578947368
testing acc= 0.757894736842
200
loss: 0.105744
training acc= 0.881578947368
testing acc= 0.757894736842
250
loss: 0.10458
training acc= 0.881578947368
testing acc= 0.757894736842
300
loss: 0.103949
training acc= 0.881578947368
testing acc= 0.757894736842
350
loss: 0.103616
training acc= 0.881578947368
testing acc= 0.757894736842
400
loss: 0.103392
training acc= 0.881578947368
testing acc= 0.757894736842
450
loss: 0.10316
training acc= 0.881578947368
testing acc= 0.757894736842
500
loss: 0.102866
training acc= 0.881578947368
testing acc= 0.757894736842
550
loss: 0.102532
training acc= 0.881578947368
testing acc= 0.757894736842
600
loss: 0.102138
training acc= 0.881578947368
testing acc= 0.757894736842
650
loss: 0.101664

In [13]:
TP = 0
FP = 0
for i in xrange(0,len(test_Y)):
    if test_Y[i]==1 and test_Y[i]==test_prediction[i]:
        TP = TP + 1
    elif test_Y[i]==0 and test_prediction[i]==1:
        FP = FP+1
TP = float(TP) / float((TP+FP))
print 'precision:',TP

precision: 0.782894736842


In [11]:
test_prediction

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      